In [23]:
import json
import csv
import os
import pandas as pd
from tqdm import tqdm

def flatten_dict(d, parent_key='', sep='_'):
    """
    Flatten nested dictionaries, creating composite keys with separator
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            # Handle lists by joining elements with a semicolon
            if all(isinstance(x, (str, int, float)) for x in v):
                items.append((new_key, ';'.join(str(x) for x in v)))
            elif len(v) > 0 and isinstance(v[0], dict):
                # For list of dictionaries, take the first one
                items.extend(flatten_dict(v[0], new_key, sep=sep).items())
            else:
                items.append((new_key, str(v)))
        else:
            items.append((new_key, v))
    return dict(items)

def process_json_files(input_dir, output_file):
    """
    Process all JSON files in the input directory and create a single CSV file
    """
    # Get list of JSON files
    json_files = [f for f in os.listdir(input_dir) if f.endswith('.json')]
    all_records = []
    all_keys = set()
    
    print("Reading JSON files and collecting fields...")
    # First pass: collect all possible fields
    for json_file in tqdm(json_files):
        file_path = os.path.join(input_dir, json_file)
        with open(file_path, 'r') as f:
            data = json.load(f)
            if 'results' in data:
                for record in data['results']:
                    flat_record = flatten_dict(record)
                    all_keys.update(flat_record.keys())
                    all_records.append(flat_record)
    
    print(f"Total records collected: {len(all_records)}")
    print(f"Total unique fields: {len(all_keys)}")
    
    # Convert to DataFrame
    print("Converting to DataFrame...")
    df = pd.DataFrame(all_records)
    
    # Basic data cleaning
    df = df.replace({None: pd.NA, "": pd.NA})
    
    # Save to CSV
    print(f"Saving to {output_file}...")
    df.to_csv(output_file, index=False)
    
    # Print some basic statistics
    print("\nDataset Statistics:")
    print(f"Total rows: {len(df)}")
    print(f"Total columns: {len(df.columns)}")
    print("\nSample columns:")
    for col in list(df.columns)[:10]:
        non_null = df[col].count()
        print(f"{col}: {non_null} non-null values ({(non_null/len(df))*100:.1f}% complete)")
    
    return df

# Process all files
input_dir = "fda_data/2004q3"
output_file = "fda_data/2004q3_combined.csv"

print("Starting conversion process...")
df = process_json_files(input_dir, output_file)
print(f"\nConversion complete. Output saved to: {output_file}")

Starting conversion process...
Reading JSON files and collecting fields...


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:25<00:00,  5.10s/it]


Total records collected: 54038
Total unique fields: 80
Converting to DataFrame...
Saving to fda_data/2004q3_combined.csv...

Dataset Statistics:
Total rows: 54038
Total columns: 80

Sample columns:
safetyreportid: 54038 non-null values (100.0% complete)
transmissiondateformat: 54038 non-null values (100.0% complete)
transmissiondate: 54038 non-null values (100.0% complete)
serious: 54038 non-null values (100.0% complete)
receivedateformat: 54038 non-null values (100.0% complete)
receivedate: 54038 non-null values (100.0% complete)
receiptdateformat: 54038 non-null values (100.0% complete)
receiptdate: 54038 non-null values (100.0% complete)
fulfillexpeditecriteria: 54038 non-null values (100.0% complete)
companynumb: 48750 non-null values (90.2% complete)

Conversion complete. Output saved to: fda_data/2004q3_combined.csv


In [31]:
import pandas as pd

# Load the data
file_path = 'fda_data/2004q3_combined.csv'  # Update with your actual path
df = pd.read_csv(file_path)

# Display basic information about the data
print("Basic Info:")
print(df.info())

# Show the first few rows
print("\nFirst Few Rows:")
print(df.head())

# Get summary statistics for numerical columns
print("\nSummary Statistics (Numerical Columns):")
print(df.describe())

# Check for missing values
print("\nMissing Values:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# Count unique values in each column
print("\nUnique Values in Each Column:")
unique_counts = df.nunique()
print(unique_counts)

# Check column names
print("\nColumn Names:")
print(df.columns)

# Sample categorical data distribution for non-numerical columns
print("\nCategorical Data Distribution (Top 5 Columns):")
for col in df.select_dtypes(include=['object']).columns[:5]:  # Adjust the number as needed
    print(f"\n{col} Value Counts:")
    print(df[col].value_counts().head())


C:\Users\samee\AppData\Local\Temp\ipykernel_2056\53358210.py:5: DtypeWarning: Columns (60,61,64,65,68,73,74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54038 entries, 0 to 54037
Data columns (total 80 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   safetyreportid                                    54038 non-null  object 
 1   transmissiondateformat                            54038 non-null  int64  
 2   transmissiondate                                  54038 non-null  int64  
 3   serious                                           54038 non-null  int64  
 4   receivedateformat                                 54038 non-null  int64  
 5   receivedate                                       54038 non-null  int64  
 6   receiptdateformat                                 54038 non-null  int64  
 7   receiptdate                                       54038 non-null  int64  
 8   fulfillexpeditecriteria                           54038 non-null  int64  
 9   compa

In [30]:
import os
import requests
import zipfile
import time
import json
import pandas as pd
from tqdm import tqdm

# Configuration
BASE_URL = "https://download.open.fda.gov/drug/event/"
QUARTERS = {
    "2024q1": list(f"drug-event-{str(i+1).zfill(4)}-of-0030.json.zip" for i in range(30)),
    "2024q2": list(f"drug-event-{str(i+1).zfill(4)}-of-0031.json.zip" for i in range(31)),
    "2024q3": list(f"drug-event-{str(i+1).zfill(4)}-of-0034.json.zip" for i in range(34))
}

def create_directory_structure():
    """Create the necessary directory structure for data organization"""
    # Create base directories
    os.makedirs("fda_data", exist_ok=True)
    os.makedirs("converted_data", exist_ok=True)
    
    # Create quarter directories
    for quarter in QUARTERS.keys():
        os.makedirs(f"fda_data/{quarter}", exist_ok=True)
        os.makedirs(f"converted_data/{quarter}", exist_ok=True)

def download_and_extract_file(url, quarter, filename):
    """Download and extract a single FDA data file"""
    local_zip_path = f"fda_data/{quarter}/{filename}"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Accept': 'application/zip',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive'
    }
    
    try:
        response = requests.get(
            url,
            headers=headers,
            stream=True,
            allow_redirects=True,
            timeout=60
        )
        
        if response.status_code == 200:
            # Save the zip file
            with open(local_zip_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            # Extract the file
            with zipfile.ZipFile(local_zip_path, "r") as zip_ref:
                zip_ref.extractall(f"fda_data/{quarter}")
            
            # Delete the zip file after extraction
            os.remove(local_zip_path)
            return True
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")
            return False
            
    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")
        if os.path.exists(local_zip_path):
            os.remove(local_zip_path)
        return False

def flatten_dict(d, parent_key='', sep='_'):
    """Flatten nested dictionaries for CSV conversion"""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            if all(isinstance(x, (str, int, float)) for x in v):
                items.append((new_key, ';'.join(str(x) for x in v)))
            elif len(v) > 0 and isinstance(v[0], dict):
                items.extend(flatten_dict(v[0], new_key, sep=sep).items())
            else:
                items.append((new_key, str(v)))
        else:
            items.append((new_key, v))
    return dict(items)

def process_json_to_csv(quarter, json_file):
    """Convert a JSON file to CSV format"""
    input_path = f"fda_data/{quarter}/{json_file}"
    output_path = f"converted_data/{quarter}/{json_file.replace('.json', '.csv')}"
    
    try:
        with open(input_path, 'r') as f:
            data = json.load(f)
            
        if 'results' in data:
            records = []
            for record in data['results']:
                flat_record = flatten_dict(record)
                records.append(flat_record)
            
            df = pd.DataFrame(records)
            df = df.replace({None: pd.NA, "": pd.NA})
            df.to_csv(output_path, index=False)
            
            print(f"Successfully converted {json_file} to CSV")
            return True
    except Exception as e:
        print(f"Error converting {json_file}: {str(e)}")
        return False

def main():
    """Main execution function"""
    create_directory_structure()
    
    for quarter, files in QUARTERS.items():
        print(f"\nProcessing {quarter}...")
        
        for file in tqdm(files, desc="Downloading and extracting"):
            file_url = f"{BASE_URL}{quarter}/{file}"
            if download_and_extract_file(file_url, quarter, file):
                # Get the JSON filename (remove .zip extension)
                json_filename = file.replace('.zip', '')
                # Process the JSON to CSV after successful download
                process_json_to_csv(quarter, json_filename)
                
            # Add delay between requests to avoid overwhelming the server
            time.sleep(2)
            
        print(f"Completed processing {quarter}")

if __name__ == "__main__":
    main()


Processing 2024q1...


Successfully converted drug-event-0001-of-0030.json to CSV


Successfully converted drug-event-0002-of-0030.json to CSV


Successfully converted drug-event-0003-of-0030.json to CSV


Successfully converted drug-event-0004-of-0030.json to CSV


...

Successfully converted drug-event-0031-of-0034.json to CSV


Successfully converted drug-event-0032-of-0034.json to CSV


Successfully converted drug-event-0033-of-0034.json to CSV


Successfully converted drug-event-0034-of-0034.json to CSV


Completed processing 2024q3
